In [5]:
%pylab inline
import itertools
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [67]:
class IO(object):
    h1 = np.ones((4, 4))
    h1[3, 2] = 3.0 # не на багато перевищує
    h1[3, 1] = 5.0 # суттєво перевищує
    h1[3, 0] = 5.0 # суттєво перевищує
    h1[2, 1] = 3.0 # не на багато перевищує
    h1[2, 0] = 3.0 # не на багато перевищує
    h1[1, 0] = 3.0 # не на багато перевищує
    h1_names = ['-20%', '-10%', '0%', '+10%']
    
    h2_layers = [
        ["FPM", "IASA"],
        ["CS", "AM", "PI"],
        ["Programmer", "Mathematicisian", "Data Scientist", "Trader"]
    ]
    h2_layer_names = ['Faculty', 'Direction', 'Profession']
    h2_local_weights = [[0.3, 0.7], [0.33, 0.33, 0.33]]
    
def fullfill(x):
    x = atleast_2d(x)
    m, n = x.shape
    
    for i, j in itertools.product(range(m), range(n)):
        if x[i, j] != 1:
            x[j, i] = 1.0 / x[i, j]
    """
    a1 = x == 1
    a2 = x != 1
    
    x[a1] = 1.0 / x[a2]
    """
    return x
        

In [68]:
pd.DataFrame(data=fullfill(IO.h1), columns=IO.h1_names, index=IO.h1_names)

,-20%,-10%,0%,+10%
-20%,1.0,0.333333,0.333333,0.200000
-10%,3.0,1.000000,0.333333,0.200000
0%,3.0,3.000000,1.000000,0.333333
+10%,5.0,5.000000,3.000000,1.000000


# Ну низя жи так

Короче, Склифасофский, сюда надо пихнуть 2 вещи:
- EM/CR (done)
- Считывалку файлов + итеративный подсчет

In [1]:
def cr_metric(lamb, dim):
    """
    Eats matrix of pairwise comparisons m and produces CR metric of consistensy.
    """
    # calculate ci metric
    assert lamb >= dim, "Greatest eigenvalue must be greater than matrix dimension."
    ci = (lamb - dim)
    if dim > 1:
        ci /= (dim - 1)
    # fill the MRCI table
    mrci = [0, 0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.40, 1.45, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59] 
    cr = ci / mrci[m.shape[0]]
    return cr

def em_ci(mat):
    """
    Eats matrix of pairwise comparisons m and produces local weights.
    """
    assert mat.shape[0] == mat.shape[1]
    values, vectors = linalg.eig(mat)
    max_index = np.argmax(abs(values))
    v = vectors[:, max_index]
    lamb = values[max_index]
    assert (v.imag < 1e-6).all(), "eigenvector has complex coordinates: %s" % v.imag
    v = v.real
    if (v < 0).all():
        v = - v
    assert (v >= 0).all(), "eigenvector has negative coordinates: %s" % v
    return v / linalg.norm(v, ord=1), cr_metric(lamb, mat.shape[0])